In [3]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics

import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics


In [4]:
pd_loan_train = pd.read_csv("../data/pd_loan_train.csv")

pd_loan_test = pd.read_csv("../data/pd_loan_train.csv")

In [5]:
pd_loan_train.columns

Index(['SK_ID_CURR', 'Unnamed: 0', 'COMMONAREA_AVG', 'COMMONAREA_MODE',
       'COMMONAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAPARTMENTS_AVG',
       'FONDKAPREMONT_MODE', 'LIVINGAPARTMENTS_AVG',
       ...
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'NWEEKDAY_PROCESS_START',
       'TARGET'],
      dtype='object', length=123)

In [6]:
pd_loan_train.dtypes

SK_ID_CURR                  int64
Unnamed: 0                  int64
COMMONAREA_AVG            float64
COMMONAREA_MODE           float64
COMMONAREA_MEDI           float64
                           ...   
FLAG_DOCUMENT_14            int64
FLAG_DOCUMENT_20            int64
FLAG_DOCUMENT_21            int64
NWEEKDAY_PROCESS_START      int64
TARGET                      int64
Length: 123, dtype: object

CODIFICACIÓN DE LA VARIABLE OBJETIVO

Como las categorías de TARGET ya están en 0 y 1 no hacemos la codificación de la varibale objetivo, pues ya están codificadas previamente.

CODIFICACIÓN DEL RESTO DE VARIABLES

Codificación del resto de variables: usaremos One-hot Enconding, que es una técnica utilizada para transformar variables categóricas en variables numéricas binarias, creando nuevas columnas (features) para cada categoría única. Cada fila tendrá un 1 en la columna correspondiente a su categoría y 0 en el resto

In [7]:
x_train = pd_loan_train.drop('TARGET', axis=1)
x_test = pd_loan_test.drop('TARGET', axis=1)
y_train = pd_loan_train['TARGET']
y_test = pd_loan_test['TARGET']

X_train y X_test se obtienen eliminando la columna TARGET (que es la variable objetivo) de los conjuntos de entrenamientos pd_loan_train y pd_loan_test.Estas variables solo contienen las características que el modelo utilizará para hacer las predicciones, sin TARGET, pues si no, el modelo tendría acceso directo a lo que se quiere predecir. Por otra parte, y_train e y_test son las variables objetivo de los conjuntos de entrenamiento y prueba que se usn para entrenar el modelo.

In [9]:
list_columns_cat = list(x_train.select_dtypes("object", "category").columns)
list_other = list(set(x_train.columns)-set(list_columns_cat))

Aqui se seleccionan solo las columnas que son de tipo 'object' y 'category', que tienen que ser codificadas anter de ser utilizadas en el modelo. En cuanto a list_other, obtiene las demas columnas que no son catgóricas.

In [10]:
ohe = ce.OneHotEncoder(cols = list_columns_cat)
model = ohe.fit(x_train, y_train)

In [11]:
model

OneHotEncoder(cols=['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE',
                    'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'OCCUPATION_TYPE',
                    'NAME_TYPE_SUITE', 'ORGANIZATION_TYPE',
                    'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'NAME_INCOME_TYPE',
                    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                    'NAME_EDUCATION_TYPE', 'CODE_GENDER', 'FLAG_OWN_REALTY'])

Las columnas listadas en OneHotEncoder(cols=[...]) son las que se han identificado como categóricas. Estas serán transformadas en un conjunto de columnas binarias. Por ejemplo 'NAME_CONTRACT_TYPE': Puede tener categorías como 'Cash loans' y 'Revolving loans','CODE_GENDER': Puede tener categorías como 'M' (hombre) y 'F' (mujer) o'FLAG_OWN_CAR': Puede indicar si el cliente tiene un coche (e.g., 'Y' para "sí" y 'N' para "no").

In [13]:
x_train_t = model.transform(x_train, y_train)
x_test_t = model.transform(x_test, y_test)

Aplicamos la transformación OneHotEncoding al conjunto de datos de entrenamiento y de prueba que toma las variables categóricas y crea unas nuevas columnas binarias para cada categoría única. La variable resultante es X_train_t donde las variables categóricas han sido transformadas en columnas con valores 0 y 1.

In [14]:
len(list(x_train_t.columns))

246

In [15]:
x_train_t.dtypes.to_dict()

{'SK_ID_CURR': dtype('int64'),
 'Unnamed: 0': dtype('int64'),
 'COMMONAREA_AVG': dtype('float64'),
 'COMMONAREA_MODE': dtype('float64'),
 'COMMONAREA_MEDI': dtype('float64'),
 'NONLIVINGAPARTMENTS_MEDI': dtype('float64'),
 'NONLIVINGAPARTMENTS_MODE': dtype('float64'),
 'NONLIVINGAPARTMENTS_AVG': dtype('float64'),
 'FONDKAPREMONT_MODE_1': dtype('int64'),
 'FONDKAPREMONT_MODE_2': dtype('int64'),
 'FONDKAPREMONT_MODE_3': dtype('int64'),
 'FONDKAPREMONT_MODE_4': dtype('int64'),
 'FONDKAPREMONT_MODE_5': dtype('int64'),
 'LIVINGAPARTMENTS_AVG': dtype('float64'),
 'LIVINGAPARTMENTS_MEDI': dtype('float64'),
 'LIVINGAPARTMENTS_MODE': dtype('float64'),
 'FLOORSMIN_MODE': dtype('float64'),
 'FLOORSMIN_AVG': dtype('float64'),
 'FLOORSMIN_MEDI': dtype('float64'),
 'YEARS_BUILD_AVG': dtype('float64'),
 'YEARS_BUILD_MODE': dtype('float64'),
 'YEARS_BUILD_MEDI': dtype('float64'),
 'OWN_CAR_AGE': dtype('float64'),
 'LANDAREA_MEDI': dtype('float64'),
 'LANDAREA_AVG': dtype('float64'),
 'LANDAREA_MODE': 

El código X_train_t.dtypes.to_dict() tiene como objetivo obtener un diccionario que mapea los nombres de las columnas de un DataFrame de pandas (X_train_t) a sus respectivos tipos de datos (dtypes).

ESCALADO DE VARIABLES

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(x_train_t)
x_train_scaled = pd.DataFrame(scaler.transform(x_train_t), columns=x_train_t.columns, index=x_train_t.index)
x_test_scaled = pd.DataFrame(scaler.transform(x_test_t), columns=x_test_t.columns, index=x_test.index)

En esta línea se importa StandardScaler del módulo sklearn.preprocessing. StandardScaler es una herramienta que permite estandarizar (escalar) las características de los datos haciendo que cada característica tenga una media de 0 y una desviación estándar de 1. Esta transformación es muy útil cuando las variables tienen rangos muy diferentes entre sí. Creamos una instancia para StandScaler con el nombre scaler que se utilizará para ajustar los datos de entrenamiento (fit()) y para transformar tanto los datos de entrenamiento como los de prueba (transform()). El método .fit() se utiliza para calcular la media y desviación estándar de las características en los datos de entrenamiento X_train_t. Esta información se almacenará en el objeto scaler y será utilizada para escalar tanto los datos de entrenamiento como los de prueba. Aquí aplicamos la normalización a los datos de entrenamiento de X_train_t utilizando los valores de media y desviación típica calculados con fit(). De esta manera, cada columna de X_train_t se reescalará para tener media 0 y desviación estándar de 1. Esto se hace tanto para X_train_t y para X_test_t. Posteriormente, se pasa a un Data Set de pandas para mantener la estructura original, manteniendo tanto las columnas como el índice

In [18]:
x_train_scaled.describe()

,SK_ID_CURR,Unnamed: 0,COMMONAREA_AVG,COMMONAREA_MODE,COMMONAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_AVG,FONDKAPREMONT_MODE_1,FONDKAPREMONT_MODE_2,...,FLAG_DOCUMENT_13,FLAG_DOCUMENT_19,FLAG_DOCUMENT_18,FLAG_DOCUMENT_17,FLAG_DOCUMENT_16,FLAG_DOCUMENT_15,FLAG_DOCUMENT_14,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,NWEEKDAY_PROCESS_START
count,2.460080e+05,2.460080e+05,7.411100e+04,7.411100e+04,7.411100e+04,7.513200e+04,7.513200e+04,7.513200e+04,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,-1.882877e-16,-1.437214e-16,-6.894645e-17,-1.893541e-18,-4.013587e-17,-4.256948e-17,2.376136e-18,-1.361845e-17,-3.865978e-17,9.805749e-18,...,-2.156831e-17,1.944542e-17,-2.469489e-17,-2.321464e-18,5.083392e-18,1.092496e-17,4.388758e-17,3.899193e-18,-7.971684e-18,7.440238e-17
std,1.000002e+00,1.000002e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.732525e+00,-1.731148e+00,-5.912407e-01,-5.761840e-01,-5.901483e-01,-1.809914e-01,-1.731877e-01,-1.828386e-01,-1.362086e-01,-5.612882e-01,...,-5.933267e-02,-2.369118e-02,-9.101224e-02,-1.662799e-02,-1.006877e-01,-3.494226e-02,-5.360967e-02,-2.272686e-02,-1.826017e-02,-1.410937e+00
25%,-8.669221e-01,-8.668357e-01,-4.873374e-01,-4.781671e-01,-4.850512e-01,-1.809914e-01,-1.731877e-01,-1.828386e-01,-1.362086e-01,-5.612882e-01,...,-5.933267e-02,-2.369118e-02,-9.101224e-02,-1.662799e-02,-1.006877e-01,-3.494226e-02,-5.360967e-02,-2.272686e-02,-1.826017e-02,-8.525904e-01
50%,5.631267e-04,3.317280e-04,-3.128330e-01,-3.188896e-01,-3.147673e-01,-1.809914e-01,-1.731877e-01,-1.828386e-01,-1.362086e-01,-5.612882e-01,...,-5.933267e-02,-2.369118e-02,-9.101224e-02,-1.662799e-02,-1.006877e-01,-3.494226e-02,-5.360967e-02,-2.272686e-02,-1.826017e-02,-2.942437e-01
75%,8.640654e-01,8.646047e-01,9.079156e-02,8.815300e-02,9.098729e-02,-9.973757e-02,-8.982641e-02,-1.021606e-01,-1.362086e-01,-5.612882e-01,...,-5.933267e-02,-2.369118e-02,-9.101224e-02,-1.662799e-02,-1.006877e-01,-3.494226e-02,-5.360967e-02,-2.272686e-02,-1.826017e-02,8.224498e-01
max,1.732494e+00,1.732127e+00,1.272970e+01,1.303728e+01,1.271328e+01,2.065332e+01,2.120149e+01,2.050381e+01,7.341679e+00,1.781616e+00,...,1.685412e+01,4.220979e+01,1.098753e+01,6.013954e+01,9.931703e+00,2.861864e+01,1.865335e+01,4.400081e+01,5.476402e+01,1.939143e+00


CONCLUSIÓN DEL ANÁLISIS EDA REALIZADO:

Respondiendo a la pregunta objetivo de si hay algún tipo de clientes más propensos al impago de un préstamos, la respuesta es sí. De acuerdo con el análisis realizado, algunos perfiles de clientes son más propensos a incumplir con el pago de un préstamo. Los factores más significativos incluyen la situación laboral, el nivel educativo y el tipo de trabajo. Por ejemplo, los desempleados y las personas en baja por maternidad tienen mayores probabilidades de incumplimiento, mientras que los trabajadores empleados y pensionistas presentan un menor riesgo debido a la estabilidad de sus ingresos.

El nivel educativo también desempeña un papel importante, ya que los clientes con niveles bajos de formación, como educación secundaria o inferior, tienen mayor probabilidad de impago, posiblemente debido a oportunidades laborales más limitadas. Por el contrario, aquellos con educación superior muestran menor riesgo.

El tipo de ocupación refuerza esta tendencia: trabajadores manuales o de baja especialización, como obreros o agricultores, son más propensos a incumplir, mientras que roles más especializados o cualificados presentan menores tasas de impago.

Adicionalmente, las puntuaciones de riesgo crediticio externas (EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3) se destacan como variables predictivas clave. Clientes con valores bajos en estas puntuaciones tienen un riesgo significativamente mayor, mientras que valores altos se asocian con un historial más confiable.

Finalmente, otros factores, como el tipo de vivienda o el género, tienen una relación más moderada, pero destacan tendencias como un mayor incumplimiento entre clientes que viven con sus padres o de alquiler, y una ligera mayor propensión entre hombres frente a mujeres. En conclusión, el riesgo de impago está influido principalmente por características relacionadas con la estabilidad laboral, educativa y económica.